# Extract, Trandform, Load

### Install Libraries & Import Libraries

In [1]:
!pip install azure-storage-blob # Microoft Azure
!pip install pyarrow
!pip install psycopg2 sqlalchemy

In [2]:
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [3]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    blob_content = download_stream.readall().decode('utf-8')
    return blob_content

### Information Architecture
Source 1 --> Gather Data --> Convert to DataFrame --> Clean Data (delete columns with less than 60% of the data count) --> Upload to Cloud (Azure)

Download (Extract) from Azure --> Reformat Data --> Transform Data (create Dimension tables and Fact table) --> Load to Data Warehouse

### Download dataframe from Azure

In [4]:
config_file_path = 'config/config.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

connection_string = config["connection_string"]
container_name = "waterconsumption"
blob_name = "WaterConsumption.csv"

blob_content = azure_download_blob(connection_string, container_name, blob_name)
water_df = pd.read_csv(StringIO(blob_content))
water_df

,development_name,borough,account_name,location,meter_amr,tds,edp,rc_code,funding_source,amp,...,service_end_date,days,meter_number,estimated,current_charges,rate_class,bill_analyzed,consumption_hcf,water_sewer_charges,other_charges
0,FHA REPOSSESSED HOUSES (GROUP V),FHA,FHA REPOSSESSED HOUSES (GROUP V),117-11 192ND STREET,NONE,260.0,325,Q026000,FEDERAL,NY005012090P,...,2023-01-19T00:00:00.000,30.0,V52311732,N,38.07,Basic Water and Sewer,Yes,204,0.00,38.07
1,VAN DYKE I,BROOKLYN,VAN DYKE I,BLD 24 - Community Center,AMR,61.0,325,K006100,FEDERAL,NY005000610P,...,2023-01-27T00:00:00.000,29.0,V84003679,Y,111.37,Basic Water and Sewer,Yes,204,111.37,0.00
2,FHA REPOSSESSED HOUSES (GROUP II),FHA,FHA REPOSSESSED HOUSES (GROUP II),171-28 111TH AVENUE,AMR,212.0,325,Q021200,FEDERAL,NY005012090P,...,2023-01-19T00:00:00.000,30.0,K15842657,N,38.07,Basic Water and Sewer,Yes,204,38.07,0.00
3,SOUTH JAMAICA II,QUEENS,SOUTH JAMAICA II,BLD 15,AMR,66.0,325,Q006600,FEDERAL,NY005010080P,...,2023-01-19T00:00:00.000,30.0,E132272916,N,2962.44,Basic Water and Sewer,Yes,204,2962.44,0.00
4,FHA REPOSSESSED HOUSES (GROUP X),FHA,FHA REPOSSESSED HOUSES (GROUP X),NaN,NaN,284.0,325,Q028400,FEDERAL,NaN,...,2023-01-27T00:00:00.000,29.0,K96779329,Y,256.15,NaN,Yes,204,256.15,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50310,SAMUEL (CITY),MANHATTAN,SAMUEL (CITY),BLD 17,AMR,377.0,650,M037700,MIXED FINANCE/LLC1,NY005023770P,...,2013-01-17T00:00:00.000,92.0,K17794667,N,53286.43,Basic Water and Sewer,Yes,6069,53286.43,0.00
50311,STANTON STREET,MANHATTAN,STANTON STREET,BLD 01,AMR,559.0,841,M055900,FEDERAL,NY005013590P,...,2013-02-13T00:00:00.000,91.0,E11337376,N,2914.99,Basic Water and Sewer,Yes,332,2914.99,0.00
50312,FHA REPOSSESSED HOUSES (GROUP VII),FHA,FHA REPOSSESSED HOUSES (GROUP VII),129-37 132ND STREET,AMR,274.0,396,Q027400,FEDERAL,NY005012090P,...,2013-02-13T00:00:00.000,91.0,V48018462,N,108.42,Basic Water and Sewer,Yes,14,108.42,0.00
50313,FHA REPOSSESSED HOUSES (GROUP V),FHA,ZZ_SOLD_FHA REPOSSESSED HOUSES (GROUP V),NaN,NONE,260.0,376,Q026000,FEDERAL,NY005012090P,...,2013-01-17T00:00:00.000,92.0,N32038857,Y,109.61,Basic Water and Sewer,Yes,3,109.61,0.00


### Transformation

In [5]:
water_df.columns

Index(['development_name', 'borough', 'account_name', 'location', 'meter_amr',
       'tds', 'edp', 'rc_code', 'funding_source', 'amp', 'vendor_name',
       'umis_bill_id', 'revenue_month', 'service_start_date',
       'service_end_date', 'days', 'meter_number', 'estimated',
       'current_charges', 'rate_class', 'bill_analyzed', 'consumption_hcf',
       'water_sewer_charges', 'other_charges'],
      dtype='object')

In [6]:
water_df.dropna(subset=['development_name'], inplace=True)
water_df.dropna(subset=['days'], inplace=True)
water_df['days'] = water_df['days'].astype('Int64')
water_df = water_df[water_df['current_charges'] >= 0]
water_df = water_df[water_df['consumption_hcf'] >= 0]
water_df = water_df[water_df['days'] >= 0]

water_df

,development_name,borough,account_name,location,meter_amr,tds,edp,rc_code,funding_source,amp,...,service_end_date,days,meter_number,estimated,current_charges,rate_class,bill_analyzed,consumption_hcf,water_sewer_charges,other_charges
0,FHA REPOSSESSED HOUSES (GROUP V),FHA,FHA REPOSSESSED HOUSES (GROUP V),117-11 192ND STREET,NONE,260.0,325,Q026000,FEDERAL,NY005012090P,...,2023-01-19T00:00:00.000,30,V52311732,N,38.07,Basic Water and Sewer,Yes,204,0.00,38.07
1,VAN DYKE I,BROOKLYN,VAN DYKE I,BLD 24 - Community Center,AMR,61.0,325,K006100,FEDERAL,NY005000610P,...,2023-01-27T00:00:00.000,29,V84003679,Y,111.37,Basic Water and Sewer,Yes,204,111.37,0.00
2,FHA REPOSSESSED HOUSES (GROUP II),FHA,FHA REPOSSESSED HOUSES (GROUP II),171-28 111TH AVENUE,AMR,212.0,325,Q021200,FEDERAL,NY005012090P,...,2023-01-19T00:00:00.000,30,K15842657,N,38.07,Basic Water and Sewer,Yes,204,38.07,0.00
3,SOUTH JAMAICA II,QUEENS,SOUTH JAMAICA II,BLD 15,AMR,66.0,325,Q006600,FEDERAL,NY005010080P,...,2023-01-19T00:00:00.000,30,E132272916,N,2962.44,Basic Water and Sewer,Yes,204,2962.44,0.00
4,FHA REPOSSESSED HOUSES (GROUP X),FHA,FHA REPOSSESSED HOUSES (GROUP X),NaN,NaN,284.0,325,Q028400,FEDERAL,NaN,...,2023-01-27T00:00:00.000,29,K96779329,Y,256.15,NaN,Yes,204,256.15,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50310,SAMUEL (CITY),MANHATTAN,SAMUEL (CITY),BLD 17,AMR,377.0,650,M037700,MIXED FINANCE/LLC1,NY005023770P,...,2013-01-17T00:00:00.000,92,K17794667,N,53286.43,Basic Water and Sewer,Yes,6069,53286.43,0.00
50311,STANTON STREET,MANHATTAN,STANTON STREET,BLD 01,AMR,559.0,841,M055900,FEDERAL,NY005013590P,...,2013-02-13T00:00:00.000,91,E11337376,N,2914.99,Basic Water and Sewer,Yes,332,2914.99,0.00
50312,FHA REPOSSESSED HOUSES (GROUP VII),FHA,FHA REPOSSESSED HOUSES (GROUP VII),129-37 132ND STREET,AMR,274.0,396,Q027400,FEDERAL,NY005012090P,...,2013-02-13T00:00:00.000,91,V48018462,N,108.42,Basic Water and Sewer,Yes,14,108.42,0.00
50313,FHA REPOSSESSED HOUSES (GROUP V),FHA,ZZ_SOLD_FHA REPOSSESSED HOUSES (GROUP V),NaN,NONE,260.0,376,Q026000,FEDERAL,NY005012090P,...,2013-01-17T00:00:00.000,92,N32038857,Y,109.61,Basic Water and Sewer,Yes,3,109.61,0.00


#### Create Dimentional Tables

In [7]:
# Development Table
unique_dep_name = water_df['development_name'].unique()
unique_dep_id = list(range(1, len(unique_dep_name)+1))
dim_development_df = pd.DataFrame(zip(unique_dep_id, unique_dep_name), columns=['development_id', 'development_name'])

dim_development_df.head(200)

,development_id,development_name
0,1,FHA REPOSSESSED HOUSES (GROUP V)
1,2,VAN DYKE I
2,3,FHA REPOSSESSED HOUSES (GROUP II)
3,4,SOUTH JAMAICA II
4,5,FHA REPOSSESSED HOUSES (GROUP X)
...,...,...
105,106,FRANKLIN AVENUE III CONVENTIONAL
106,107,SAMUEL (MHOP) II
107,108,BRYANT AVENUE-EAST 174TH STREET
108,109,EAST 152ND STREET-COURTLANDT AVENUE


In [8]:
# Funding Table
unique_funding_name = water_df['funding_source'].unique()
unique_funding_name = np.concatenate((unique_funding_name[~pd.isnull(unique_funding_name)], unique_funding_name[pd.isnull(unique_funding_name)]))
unique_funding_id = list(range(1, len(unique_funding_name) + 1))
unique_funding_id[-1] = 9999
dim_funding_df = pd.DataFrame(zip(unique_funding_id, unique_funding_name), columns=['funding_id', 'funding_source'])

dim_funding_df

,funding_id,funding_source
0,1,FEDERAL
1,2,MIXED FINANCE/LLC1
2,3,MHOP
3,4,MIXED FINANCE/LLC2
4,5,SECTION 8
5,9999,NaN


In [9]:
# Vendor Table
unique_vendor_name = water_df['vendor_name'].unique()
unique_vendor_id = range(1, len(unique_vendor_name) + 1)
dim_vendor_df = pd.DataFrame(zip(unique_vendor_id, unique_vendor_name), columns=['vendor_id', 'vendor_name'])

dim_vendor_df

,vendor_id,vendor_name
0,1,NEW YORK CITY WATER BOARD


In [10]:
# Rate Class Table
unique_rate_name = water_df['rate_class'].unique()
unique_rate_name = np.concatenate((unique_rate_name[~pd.isnull(unique_rate_name)], unique_rate_name[pd.isnull(unique_rate_name)]))
unique_rate_id = list(range(1, len(unique_rate_name) + 1))
unique_rate_id[-1] = 9999
dim_rate_df = pd.DataFrame(zip(unique_rate_id, unique_rate_name), columns=['rateclass_id', 'rate_class'])

dim_rate_df

,rateclass_id,rate_class
0,1,Basic Water and Sewer
1,2,MULTIFAMILY
2,3,HOT OR COLD WATER IN STORE
3,4,COMMERCIAL
4,5,WATER-METER
5,9999,NaN


In [11]:
# Location Table
unique_location = water_df[['borough', 'location']].copy()
unique_location.drop_duplicates(subset=['borough', 'location'], inplace=True)
unique_location

,borough,location
0,FHA,117-11 192ND STREET
1,BROOKLYN,BLD 24 - Community Center
2,FHA,171-28 111TH AVENUE
3,QUEENS,BLD 15
4,FHA,NaN
...,...,...
48619,FHA,BLD 04
49386,BROOKLYN,BLD 37
49916,BROOKLYN,NaN
49966,BROOKLYN,BLD16 - STORE 35-36


In [12]:
def split_address_street(address):
  if not isinstance(address, str) or not address[0].isdigit():
    return None
  else:
    space_index = address.find(' ')
    if space_index != -1:
      street = address[space_index+1:]
      return street
    else:
      return None

In [13]:
unique_location['street'] = unique_location['location'].apply(lambda x: pd.Series(split_address_street(x)))
unique_location

,borough,location,street
0,FHA,117-11 192ND STREET,192ND STREET
1,BROOKLYN,BLD 24 - Community Center,NaN
2,FHA,171-28 111TH AVENUE,111TH AVENUE
3,QUEENS,BLD 15,NaN
4,FHA,NaN,NaN
...,...,...,...
48619,FHA,BLD 04,NaN
49386,BROOKLYN,BLD 37,NaN
49916,BROOKLYN,NaN,NaN
49966,BROOKLYN,BLD16 - STORE 35-36,NaN


In [14]:
unique_location_id = list(range(1, len(unique_location) + 1))
unique_location.insert(0, 'location_id', unique_location_id)
dim_location_df = unique_location[['location_id', 'borough', 'street', 'location']].copy()

dim_location_df

,location_id,borough,street,location
0,1,FHA,192ND STREET,117-11 192ND STREET
1,2,BROOKLYN,NaN,BLD 24 - Community Center
2,3,FHA,111TH AVENUE,171-28 111TH AVENUE
3,4,QUEENS,NaN,BLD 15
4,5,FHA,NaN,NaN
...,...,...,...,...
48619,367,FHA,NaN,BLD 04
49386,368,BROOKLYN,NaN,BLD 37
49916,369,BROOKLYN,NaN,NaN
49966,370,BROOKLYN,NaN,BLD16 - STORE 35-36


In [15]:
# Create Date Table
def week_of_month(dt):
    first_day = dt.replace(day=1)
    dom = dt.day
    adjusted_dom = dom + first_day.weekday()
    return int(ceil(adjusted_dom/7.0))

def get_week_of_year(date_str):
    """
    Calculate the ISO week number of the year for a given date.

    Parameters:
    date_str (str): A date string in the format 'YYYY-MM-DD'.

    Returns:
    int: ISO week number of the year.
    """
    # Parse the input string to a datetime object
    date = datetime.strptime(date_str, '%Y-%m-%d')

    # Get the ISO calendar week number
    week_of_year = date.isocalendar()[1]

    return week_of_year

def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day

    cal = calendar.monthcalendar(year, month)
    week_number = (day - 1) // 7 + 1
    return week_number

In [16]:
water_df['service_start_date'] = pd.to_datetime(water_df['service_start_date'])
water_df['service_end_date'] = pd.to_datetime(water_df['service_end_date'])

In [17]:
start_date = min(water_df['service_start_date'].min(), water_df['service_end_date'].min())
end_date = max(water_df['service_start_date'].max(), water_df['service_end_date'].max())
print(start_date, end_date)

2002-12-22 00:00:00 2023-01-27 00:00:00


In [18]:
dim_date_df = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='D')})
dim_date_df.head()

,date
0,2002-12-22
1,2002-12-23
2,2002-12-24
3,2002-12-25
4,2002-12-26


In [19]:
# Extract attributes
dim_date_df['date_id'] = dim_date_df['date'].dt.strftime('%Y%m%d')
dim_date_df['date_iso_format'] = dim_date_df['date'].apply(lambda x: x.isoformat()[:10])

dim_date_df['year_number'] = dim_date_df['date'].dt.year
dim_date_df['quarter_number'] = dim_date_df['date'].dt.quarter
dim_date_df['month_number'] = dim_date_df['date'].dt.month
dim_date_df['day_number'] = dim_date_df['date'].dt.day

dim_date_df['month_name'] = dim_date_df['date'].dt.strftime('%B')
dim_date_df['day_name'] = dim_date_df['date'].dt.strftime('%A')

# Add week of the month and week of the year
dim_date_df['week_of_the_year'] = dim_date_df['date'].dt.strftime('%U')
dim_date_df['week_of_the_month'] = dim_date_df['date'].apply(week_of_month)

dim_date_df.head(10)

,date,date_id,date_iso_format,year_number,quarter_number,month_number,day_number,month_name,day_name,week_of_the_year,week_of_the_month
0,2002-12-22,20021222,2002-12-22,2002,4,12,22,December,Sunday,51,4
1,2002-12-23,20021223,2002-12-23,2002,4,12,23,December,Monday,51,4
2,2002-12-24,20021224,2002-12-24,2002,4,12,24,December,Tuesday,51,4
3,2002-12-25,20021225,2002-12-25,2002,4,12,25,December,Wednesday,51,4
4,2002-12-26,20021226,2002-12-26,2002,4,12,26,December,Thursday,51,4
5,2002-12-27,20021227,2002-12-27,2002,4,12,27,December,Friday,51,4
6,2002-12-28,20021228,2002-12-28,2002,4,12,28,December,Saturday,51,4
7,2002-12-29,20021229,2002-12-29,2002,4,12,29,December,Sunday,52,5
8,2002-12-30,20021230,2002-12-30,2002,4,12,30,December,Monday,52,5
9,2002-12-31,20021231,2002-12-31,2002,4,12,31,December,Tuesday,52,5


### Link each other and Create Fact Table

In [20]:
fact_waterconsumption_df = water_df.copy()
fact_waterconsumption_df

,development_name,borough,account_name,location,meter_amr,tds,edp,rc_code,funding_source,amp,...,service_end_date,days,meter_number,estimated,current_charges,rate_class,bill_analyzed,consumption_hcf,water_sewer_charges,other_charges
0,FHA REPOSSESSED HOUSES (GROUP V),FHA,FHA REPOSSESSED HOUSES (GROUP V),117-11 192ND STREET,NONE,260.0,325,Q026000,FEDERAL,NY005012090P,...,2023-01-19,30,V52311732,N,38.07,Basic Water and Sewer,Yes,204,0.00,38.07
1,VAN DYKE I,BROOKLYN,VAN DYKE I,BLD 24 - Community Center,AMR,61.0,325,K006100,FEDERAL,NY005000610P,...,2023-01-27,29,V84003679,Y,111.37,Basic Water and Sewer,Yes,204,111.37,0.00
2,FHA REPOSSESSED HOUSES (GROUP II),FHA,FHA REPOSSESSED HOUSES (GROUP II),171-28 111TH AVENUE,AMR,212.0,325,Q021200,FEDERAL,NY005012090P,...,2023-01-19,30,K15842657,N,38.07,Basic Water and Sewer,Yes,204,38.07,0.00
3,SOUTH JAMAICA II,QUEENS,SOUTH JAMAICA II,BLD 15,AMR,66.0,325,Q006600,FEDERAL,NY005010080P,...,2023-01-19,30,E132272916,N,2962.44,Basic Water and Sewer,Yes,204,2962.44,0.00
4,FHA REPOSSESSED HOUSES (GROUP X),FHA,FHA REPOSSESSED HOUSES (GROUP X),NaN,NaN,284.0,325,Q028400,FEDERAL,NaN,...,2023-01-27,29,K96779329,Y,256.15,NaN,Yes,204,256.15,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50310,SAMUEL (CITY),MANHATTAN,SAMUEL (CITY),BLD 17,AMR,377.0,650,M037700,MIXED FINANCE/LLC1,NY005023770P,...,2013-01-17,92,K17794667,N,53286.43,Basic Water and Sewer,Yes,6069,53286.43,0.00
50311,STANTON STREET,MANHATTAN,STANTON STREET,BLD 01,AMR,559.0,841,M055900,FEDERAL,NY005013590P,...,2013-02-13,91,E11337376,N,2914.99,Basic Water and Sewer,Yes,332,2914.99,0.00
50312,FHA REPOSSESSED HOUSES (GROUP VII),FHA,FHA REPOSSESSED HOUSES (GROUP VII),129-37 132ND STREET,AMR,274.0,396,Q027400,FEDERAL,NY005012090P,...,2013-02-13,91,V48018462,N,108.42,Basic Water and Sewer,Yes,14,108.42,0.00
50313,FHA REPOSSESSED HOUSES (GROUP V),FHA,ZZ_SOLD_FHA REPOSSESSED HOUSES (GROUP V),NaN,NONE,260.0,376,Q026000,FEDERAL,NY005012090P,...,2013-01-17,92,N32038857,Y,109.61,Basic Water and Sewer,Yes,3,109.61,0.00


In [21]:
fact_waterconsumption_df = fact_waterconsumption_df[['current_charges', 'consumption_hcf', 'water_sewer_charges', 'other_charges', 'days', 'borough', 'location', 'development_name', 'funding_source', 'vendor_name', 'rate_class', 'service_start_date', 'service_end_date']].copy()
fact_waterconsumption_df

,current_charges,consumption_hcf,water_sewer_charges,other_charges,days,borough,location,development_name,funding_source,vendor_name,rate_class,service_start_date,service_end_date
0,38.07,204,0.00,38.07,30,FHA,117-11 192ND STREET,FHA REPOSSESSED HOUSES (GROUP V),FEDERAL,NEW YORK CITY WATER BOARD,Basic Water and Sewer,2022-12-20,2023-01-19
1,111.37,204,111.37,0.00,29,BROOKLYN,BLD 24 - Community Center,VAN DYKE I,FEDERAL,NEW YORK CITY WATER BOARD,Basic Water and Sewer,2022-12-29,2023-01-27
2,38.07,204,38.07,0.00,30,FHA,171-28 111TH AVENUE,FHA REPOSSESSED HOUSES (GROUP II),FEDERAL,NEW YORK CITY WATER BOARD,Basic Water and Sewer,2022-12-20,2023-01-19
3,2962.44,204,2962.44,0.00,30,QUEENS,BLD 15,SOUTH JAMAICA II,FEDERAL,NEW YORK CITY WATER BOARD,Basic Water and Sewer,2022-12-20,2023-01-19
4,256.15,204,256.15,0.00,29,FHA,NaN,FHA REPOSSESSED HOUSES (GROUP X),FEDERAL,NEW YORK CITY WATER BOARD,NaN,2022-12-29,2023-01-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50310,53286.43,6069,53286.43,0.00,92,MANHATTAN,BLD 17,SAMUEL (CITY),MIXED FINANCE/LLC1,NEW YORK CITY WATER BOARD,Basic Water and Sewer,2012-10-17,2013-01-17
50311,2914.99,332,2914.99,0.00,91,MANHATTAN,BLD 01,STANTON STREET,FEDERAL,NEW YORK CITY WATER BOARD,Basic Water and Sewer,2012-11-14,2013-02-13
50312,108.42,14,108.42,0.00,91,FHA,129-37 132ND STREET,FHA REPOSSESSED HOUSES (GROUP VII),FEDERAL,NEW YORK CITY WATER BOARD,Basic Water and Sewer,2012-11-14,2013-02-13
50313,109.61,3,109.61,0.00,92,FHA,NaN,FHA REPOSSESSED HOUSES (GROUP V),FEDERAL,NEW YORK CITY WATER BOARD,Basic Water and Sewer,2012-10-17,2013-01-17


In [22]:
merged_location_df = fact_waterconsumption_df.merge(dim_location_df, left_on=['borough', 'location'], right_on=['borough', 'location'], how='left')
fact_waterconsumption_df = merged_location_df.copy()

In [23]:
merged_development_df = fact_waterconsumption_df.merge(dim_development_df, left_on=['development_name'], right_on=['development_name'], how='left')
fact_waterconsumption_df = merged_development_df.copy()

In [24]:
merged_funding_df = fact_waterconsumption_df.merge(dim_funding_df, left_on=['funding_source'], right_on=['funding_source'], how='left')
fact_waterconsumption_df = merged_funding_df.copy()

In [25]:
merged_vendor_df = fact_waterconsumption_df.merge(dim_vendor_df, left_on=['vendor_name'], right_on=['vendor_name'], how='left')
fact_waterconsumption_df = merged_vendor_df.copy()

In [26]:
merged_rate_df = fact_waterconsumption_df.merge(dim_rate_df, left_on=['rate_class'], right_on=['rate_class'], how='left')
fact_waterconsumption_df = merged_rate_df.copy()

In [27]:
merged_startdate_df = fact_waterconsumption_df.merge(dim_date_df, left_on=['service_start_date'], right_on=['date'], how='left')
fact_waterconsumption_df = merged_startdate_df.copy()

In [28]:
merged_enddate_df = fact_waterconsumption_df.merge(dim_date_df, left_on=['service_end_date'], right_on=['date'], how='left')
fact_waterconsumption_df = merged_enddate_df.copy()

In [29]:
fact_waterconsumption_df.columns

Index(['current_charges', 'consumption_hcf', 'water_sewer_charges',
       'other_charges', 'days', 'borough', 'location', 'development_name',
       'funding_source', 'vendor_name', 'rate_class', 'service_start_date',
       'service_end_date', 'location_id', 'street', 'development_id',
       'funding_id', 'vendor_id', 'rateclass_id', 'date_x', 'date_id_x',
       'date_iso_format_x', 'year_number_x', 'quarter_number_x',
       'month_number_x', 'day_number_x', 'month_name_x', 'day_name_x',
       'week_of_the_year_x', 'week_of_the_month_x', 'date_y', 'date_id_y',
       'date_iso_format_y', 'year_number_y', 'quarter_number_y',
       'month_number_y', 'day_number_y', 'month_name_y', 'day_name_y',
       'week_of_the_year_y', 'week_of_the_month_y'],
      dtype='object')

In [30]:
fact_waterconsumption_df_final = fact_waterconsumption_df.copy()
fact_waterconsumption_df_final = fact_waterconsumption_df_final[['current_charges', 'consumption_hcf', 'water_sewer_charges',
       'other_charges', 'days', 'location_id', 'development_id',
       'funding_id', 'vendor_id',
       'rateclass_id', 'date_id_x',
       'date_id_y']]
fact_id = list(range(1, len(fact_waterconsumption_df_final) + 1))
fact_waterconsumption_df_final.insert(0, 'fact_id', fact_id)
fact_waterconsumption_df_final.rename(columns={'days': 'number_of_days', 'date_id_x': 'service_startdate_id', 'date_id_y': 'service_enddate_id'}, inplace=True)

fact_waterconsumption_df_final

,fact_id,current_charges,consumption_hcf,water_sewer_charges,other_charges,number_of_days,location_id,development_id,funding_id,vendor_id,rateclass_id,service_startdate_id,service_enddate_id
0,1,38.07,204,0.00,38.07,30,1,1,1,1,1,20221220,20230119
1,2,111.37,204,111.37,0.00,29,2,2,1,1,1,20221229,20230127
2,3,38.07,204,38.07,0.00,30,3,3,1,1,1,20221220,20230119
3,4,2962.44,204,2962.44,0.00,30,4,4,1,1,1,20221220,20230119
4,5,256.15,204,256.15,0.00,29,5,5,1,1,9999,20221229,20230127
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50237,50238,53286.43,6069,53286.43,0.00,92,244,19,2,1,1,20121017,20130117
50238,50239,2914.99,332,2914.99,0.00,91,56,104,1,1,1,20121114,20130213
50239,50240,108.42,14,108.42,0.00,91,153,30,1,1,1,20121114,20130213
50240,50241,109.61,3,109.61,0.00,92,5,1,1,1,1,20121017,20130117


In [31]:
fact_waterconsumption_df_final.columns

Index(['fact_id', 'current_charges', 'consumption_hcf', 'water_sewer_charges',
       'other_charges', 'number_of_days', 'location_id', 'development_id',
       'funding_id', 'vendor_id', 'rateclass_id', 'service_startdate_id',
       'service_enddate_id'],
      dtype='object')

In [32]:
fact_waterconsumption_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50242 entries, 0 to 50241
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fact_id               50242 non-null  int64  
 1   current_charges       50242 non-null  float64
 2   consumption_hcf       50242 non-null  int64  
 3   water_sewer_charges   50242 non-null  float64
 4   other_charges         50242 non-null  float64
 5   number_of_days        50242 non-null  Int64  
 6   location_id           50242 non-null  int64  
 7   development_id        50242 non-null  int64  
 8   funding_id            50242 non-null  int64  
 9   vendor_id             50242 non-null  int64  
 10  rateclass_id          50242 non-null  int64  
 11  service_startdate_id  50242 non-null  object 
 12  service_enddate_id    50242 non-null  object 
dtypes: Int64(1), float64(3), int64(7), object(2)
memory usage: 5.0+ MB


In [33]:
# delete 'date' column in dim_date
dim_date_df = dim_date_df.drop(dim_date_df.columns[0], axis=1)

### Upload Each Table to Data Warehouse (PostgreSQL)

In [34]:
config_file_path = 'config/config_sql.json'

# Load the JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

username = config["username"]
pwd = config["pwd"]
host = config["host"]

database_url = f'postgresql://{username}:{pwd}@{host}/postgres'
database_url

# Create a SQLAlchemy engine
engine = create_engine(database_url)

In [35]:
dim_funding_df.to_csv("funding.csv",index=False)
dim_funding_df.to_sql('dim_funding', schema='waterconsumption', con=engine, if_exists='replace', index=False)

6

In [36]:
dim_rate_df.to_csv("rateclass.csv",index=False)
dim_rate_df.to_sql('dim_rate', schema='waterconsumption', con=engine, if_exists='replace', index=False)

6

In [37]:
dim_vendor_df.to_csv("vendor.csv",index=False)
dim_vendor_df.to_sql('dim_vendor', schema='waterconsumption', con=engine, if_exists='replace', index=False)

1

In [38]:
dim_development_df.to_csv("development.csv",index=False)
dim_development_df.to_sql('dim_development', schema='waterconsumption', con=engine, if_exists='replace', index=False)

110

In [39]:
dim_location_df.to_csv("location.csv",index=False)
dim_location_df.to_sql('dim_location', schema='waterconsumption', con=engine, if_exists='replace', index=False)

371

In [40]:
dim_date_df.to_csv("date.csv",index=False)
dim_date_df.to_sql('dim_date', schema='waterconsumption', con=engine, if_exists='replace', index=False)

342

In [41]:
fact_waterconsumption_df_final.to_csv("fact_waterconsumption.csv",index=False)
fact_waterconsumption_df_final.to_sql('fact_waterconsumption', schema='waterconsumption', con=engine, if_exists='replace', index=False)

242